In [3]:
import boto3
import pandas as pd

bucket = 'poc-2209-twayairport-dp'

s3 = boto3.client('s3')
obj_list = s3.list_objects(Bucket = bucket, Prefix = 'data/original')
obj_list = [i['Key'] for i in obj_list['Contents'] if '.csv' in i['Key']]

In [4]:
booking_list = [i for i in obj_list if '예약' in i]
coupon_list = [i for i in obj_list if '쿠폰' in i]
flight_list = [i for i in obj_list if '비행편정보' in i]

In [143]:
data_list = ['booking']
for data_name in data_list:
    df = pd.DataFrame()
    for path in globals()[f'{data_name}_list']:        
        print(path)
        print(" ")                       
        df = pd.concat([df, pd.read_csv(f's3://{bucket}/{path}')])
    if data_name == 'booking':
        df['year'] = df['capture_date'].str.split('-', expand = True)[0]
        df['month'] = df['capture_date'].str.split('-', expand = True)[1]        
    elif data_name == 'coupon':
        df['issuedate'] = df['issuedate'].astype('str')
        df['issuedate'] = df['issuedate'].str.replace('-','')
        df['year'] = df['issuedate'].str.slice(start=0, stop=4)
        df['month'] = df['issuedate'].str.slice(start=4, stop=6)
    elif data_name == 'flight':
        df['year'] = df['flight_departure_date'].str.split('-', expand = True)[0]
        df['month'] = df['flight_departure_date'].str.split('-', expand = True)[1]    
        df['day'] = df['flight_departure_date'].str.split('-', expand = True)[2]    
        
    df.drop_duplicates(inplace = True)
    
    for year in df['year'].unique():        
        temp_year = df[df['year'] == year].copy()
        for month in temp_year['month'].unique():            
            temp_month = temp_year[(temp_year['year'] == year)&(temp_year['month'] == month)].copy()
            if 'day' in list(df):
                for day in temp_month['day'].unique():
                    temp_day = temp_month[temp_month['day'] == day]                    
                    temp_day.drop(['year','month','day'], axis = 1,inplace = True)
                    temp_day.to_csv(f's3://poc-2209-twayairport-dp/train_data/{data_name}/raw_csv/{year}/{month}/{day}/{data_name}.csv', index=False)
                    print(f'saved data : train_data/{data_name}/raw_csv/{year}/{month}/{day}/{data_name}.csv')                    
            else:
                temp_month.drop(['year','month'], axis = 1,inplace = True)
                temp_month.to_csv(f's3://poc-2209-twayairport-dp/train_data/{data_name}/raw_csv/{year}/{month}/{data_name}.csv', index=False)
                print(f'saved data : train_data/{data_name}/raw_csv/{year}/{month}/{data_name}.csv')

data/original/예약_20170608.csv
 
data/original/예약_20180608.csv
 
data/original/예약_20220220_20220226.csv
 
data/original/예약_20220821_20220827.csv
 
data/original/예약_20220822.csv
 
saved data : train_data/booking/raw_csv/2016/12/booking.csv
saved data : train_data/booking/raw_csv/2016/11/booking.csv
saved data : train_data/booking/raw_csv/2017/02/booking.csv
saved data : train_data/booking/raw_csv/2017/03/booking.csv
saved data : train_data/booking/raw_csv/2017/01/booking.csv
saved data : train_data/booking/raw_csv/2017/04/booking.csv
saved data : train_data/booking/raw_csv/2017/06/booking.csv
saved data : train_data/booking/raw_csv/2017/05/booking.csv
saved data : train_data/booking/raw_csv/2017/07/booking.csv
saved data : train_data/booking/raw_csv/2017/08/booking.csv
saved data : train_data/booking/raw_csv/2017/09/booking.csv
saved data : train_data/booking/raw_csv/2017/11/booking.csv
saved data : train_data/booking/raw_csv/2017/12/booking.csv
saved data : train_data/booking/raw_csv/20